<a href="https://colab.research.google.com/github/julius-banda/OTP/blob/master/MyCrackV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install scikit-learn
!pip install matplotlib
!pip install tensorflow
!pip install scikit-learn
!pip install matplotlib


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import random
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Configuration
CONFIG = {
    "data_file": "/content/drive/MyDrive/rounds_data.json",
    "model_file": "/content/drive/MyDrive/round_predictor_model.keras",
    "min_rounds": 50,
    "epochs": 200,
    "batch_size": 16,
    "learning_rate": 0.001,
    "patience": 10,
    "k_folds": 5,
    "noise_factor": 0.05,
    "scale_range": (0.8, 1.2),
}

# Utility Functions
def save_data(rounds_data):
    """Save rounds data to the configured file."""
    with open(CONFIG["data_file"], "w") as f:
        json.dump(rounds_data, f)

def load_data():
    """Load rounds data from the configured file, or return an empty list if not found."""
    if os.path.exists(CONFIG["data_file"]):
        with open(CONFIG["data_file"], "r") as f:
            return json.load(f)
    return []

def classify_color(result):
    """Classify the result into a color category."""
    if 1.00 <= result <= 1.99:
        return "BLUE"
    elif 2.00 <= result <= 9.99:
        return "PURPLE"
    elif result >= 10.00:
        return "PINK"
    return "UNKNOWN"

def determine_accuracy_score(actual, predicted):
    """Determine the accuracy score based on the error between actual and predicted values."""
    error = abs(actual - predicted)
    if error < 1.0:
        return 10  # Perfect prediction
    elif error < 2.0:
        return 9
    elif error < 3.0:
        return 8
    elif error < 5.0:
        return 7
    elif error < 7.0:
        return 6
    elif error < 9.0:
        return 5
    elif error < 11.0:
        return 4
    elif error < 15.0:
        return 3
    elif error < 20.0:
        return 2
    else:
        return 1  # Very inaccurate

def plot_color_patterns(rounds_data):
    """Plot color patterns from the rounds data."""
    colors = {"BLUE": [], "PURPLE": [], "PINK": []}
    for round_data in rounds_data:
        color = round_data["color"]
        if color in colors:
            colors[color].append(round_data["result"])
    for color, results in colors.items():
        plt.scatter(range(len(results)), results, label=color)
    plt.xlabel("Index")
    plt.ylabel("Result")
    plt.legend()
    plt.show()

# Data Preparation Functions
def add_noise_to_data(X, noise_factor):
    """Add random noise to the dataset for augmentation."""
    noise = np.random.normal(0, noise_factor, X.shape)
    return X + noise

def random_scale_features(X, scale_range):
    """Scale features randomly within the given range."""
    scale_factor = np.random.uniform(scale_range[0], scale_range[1], size=(X.shape[1],))
    return X * scale_factor

def detect_outliers(data):
    """Detect outliers using the interquartile range (IQR) method."""
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (data < lower_bound) | (data > upper_bound)

def prepare_training_data(rounds_data, augment_data=False):
    """Prepare training data with optional augmentation."""
    X = []
    y = []
    for round_data in rounds_data:
        features = [round_data["round"]]
        color = round_data.get("color")
        if color == "BLUE":
            features.append(0)
        elif color == "PURPLE":
            features.append(1)
        elif color == "PINK":
            features.append(2)
        else:
            features.append(-1)
        X.append(features)
        y.append(round_data["result"])

    X = np.array(X)
    y = np.array(y)

    if augment_data:
        X = add_noise_to_data(X, CONFIG["noise_factor"])
        X = random_scale_features(X, CONFIG["scale_range"])

    outliers = detect_outliers(y)
    X = X[~outliers]
    y = y[~outliers]

    return X, y

# Feature Engineering
def compute_rolling_averages(data, window=3):
    """Compute rolling averages for the given data."""
    return np.convolve(data, np.ones(window)/window, mode='valid')

def compute_relative_changes(data):
    """Compute relative changes between consecutive data points."""
    return np.diff(data) / data[:-1]

# Model Building
def create_nn_model(input_shape):
    """Create a neural network model with the given input shape."""
    model = Sequential([
        Dense(256, input_dim=input_shape, activation=None),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dense(128, activation=None),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.4),
        Dense(64, activation=None),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.4),
        Dense(32, activation="relu"),
        Dense(1, activation="linear")
    ])
    model.compile(optimizer=Adam(learning_rate=CONFIG["learning_rate"]), loss="mse", metrics=["mae"])
    return model

# Main Function
def main():
    """Main execution function for training and prediction."""
    rounds_data = load_data()

    if not rounds_data:
        print("No previous data found. Please start entering results of initial rounds.")
        for i in range(CONFIG["min_rounds"]):
            while True:
                try:
                    print(f"Entering data for round {i + 1}...")
                    result = float(input("Enter the actual result: "))
                    break
                except ValueError:
                    print("Invalid input. Please enter a numeric value.")

            color = classify_color(result)
            rounds_data.append({"round": i + 1, "result": result, "color": color})
        save_data(rounds_data)

    if len(rounds_data) < CONFIG["min_rounds"]:
        raise ValueError(f"A minimum of {CONFIG['min_rounds']} rounds is required for prediction.")

    X, y = prepare_training_data(rounds_data, augment_data=True)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    kf = KFold(n_splits=CONFIG["k_folds"], shuffle=True, random_state=42)
    val_scores = []

    for train_index, val_index in kf.split(X_scaled):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]

        model = create_nn_model(input_shape=X_train.shape[1])

        early_stopping = EarlyStopping(monitor='val_loss', patience=CONFIG["patience"], restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)

        model.fit(X_train, y_train, epochs=CONFIG["epochs"], batch_size=CONFIG["batch_size"],
                  validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr], verbose=1)

        val_loss, val_mae = model.evaluate(X_val, y_val, verbose=1)
        val_scores.append((val_loss, val_mae))

    avg_val_loss = np.mean([score[0] for score in val_scores])
    avg_val_mae = np.mean([score[1] for score in val_scores])

    print(f"Average Validation Loss: {avg_val_loss:.4f}")
    print(f"Average Validation MAE: {avg_val_mae:.4f}")

    model.fit(X_scaled, y, epochs=CONFIG["epochs"], batch_size=CONFIG["batch_size"],
              callbacks=[early_stopping, reduce_lr], verbose=1)

    print("Starting prediction...")
    for i in range(len(rounds_data) + 1, len(rounds_data) + 11):
        features = np.array([[i, -1]])
        features_scaled = scaler.transform(features)
        prediction = model.predict(features_scaled, verbose=0)[0][0]
        predicted_color = classify_color(prediction)
        print(f"Predicted result for round {i}: {prediction:.2f} ({predicted_color})")

        while True:
            try:
                result = float(input(f"Enter the actual result for round {i}: "))
                break
            except ValueError:
                print("Invalid input. Please enter a numeric value.")

        accuracy_score = determine_accuracy_score(result, prediction)

        if 1 <= accuracy_score <= 3:
            accuracy_color = "RED"
        elif 4 <= accuracy_score <= 7:
            accuracy_color = "YELLOW"
        else:
            accuracy_color = "GREEN"

        print(f"Prediction accuracy for round {i}: {accuracy_score}/10 ({accuracy_color})")

        color = classify_color(result)
        rounds_data.append({"round": i, "result": result, "color": color})
        save_data(rounds_data)

    model.save(CONFIG["model_file"])
    print("Model training complete and saved.")

    plot_color_patterns(rounds_data)

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - loss: 6.1052 - mae: 2.0368 - val_loss: 4.4920 - val_mae: 1.9039 - learning_rate: 0.0010
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5.5517 - mae: 1.9191 - val_loss: 4.2785 - val_mae: 1.8553 - learning_rate: 0.0010
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 4.6582 - mae: 1.8553 - val_loss: 4.1115 - val_mae: 1.8146 - learning_rate: 0.0010
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3.8438 - mae: 1.6866 - val_loss: 3.9556 - val_mae: 1.7761 - learning_rate: 0.0010
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.4985 - mae: 1.2604 - val_loss: 3.8035 - val_mae: 1.7362 - learning_rate: 0.0010
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.2763 - mae: 1.2712 - val_loss: 3.6523 - val_mae: 1.6966 - learning_rate: 0.0010
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.3712 - mae: 0.9269 - val_loss: 3.4956 - val_mae: 1.6551 - learning_rate: 0.0010
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae,learning_rate.
  callback.on_epoch_end(epoch, logs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8004 - mae: 0.5437 - learning_rate: 1.2500e-04
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4648 - mae: 0.4789 - learning_rate: 1.2500e-04
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5290 - mae: 0.5056 - learning_rate: 1.2500e-04
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4942 - mae: 0.5337 - learning_rate: 1.2500e-04
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3928 - mae: 0.4450 - learning_rate: 1.2500e-04
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3862 - mae: 0.4235 - learning_rate: 1.2500e-04
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3491 - mae: 0.4287 - learning_rate: 1.2500e-04
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4947 - mae: 0.4181 - learning_rate: 1.2500e-04
Epoch 13/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4290 - mae: 0.4313 - learning_rate: 1.2500e-04
Epoch 14/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - lo